# Challenge Dreem classification d'états de sommeil

##### Auteurs : Antoine Bourgoin et Benoit Charmettant

## Introduction et remarques préliminaires
Dans le cadre du cours de machine learning, il nous était proposé de participer à un challenge organisé par la société Dreem qui développe un bandeau connecté permettant à chacun de d'analyser son sommeil. Ils ont mis à notre disposition un jeu de données issues de leur bandeau et in nous était demandé de proposer un algorithme permettant de classifier des séquences d'enregistrement des différents capteurs du bandeau selon les différents état de sommeils connus et utilisés en pratique. Le tout était proposé sous le format d'un challenge entre les étudiants sur le site spécialisé Kaggle.
Ce mode de gestion de projet ne met pas forcément l'accent la réponse à une problématique précise comme cela pourrait être le cas dans un travail de recherche plus classique. Ce rapport reflète donc ce travers et ne s'étend pas sur une analyse quantitative des différentes approches que nous avons pu envisager, ce n'était pas à notre avis l'objectif du travail. D'autant plus que, comme nous le verrons plus tard la méthode qui s'est avérée la plus efficace est une méthode assez "gloutonne" ne laissant pas beaucoup d'espace à des approches plus fines (voir la conclusion pour plus de détails). Nous avons néamoins essayé d'expliciter certaines pistes que nous avons exploré même si celles ci ne nous ont pas nécessairement permis d'obtenir de meilleurs résultats que la méthode "gloutonne". Nous tachons dans ce rapport d'exposer la démarche qui a été la notre, les différents aspects du travail que nous avons fourni, certaines pistes que nous avons exploré et essayons d'offrir une analyse critique de nos résultats.
Nous avions commencé notre travail dans un environnement RStudio et avons rapidement migré vers des notebook Python du fait de la taille des données que nous avions du mal à gérer sur R. Etant plus à l'aise sur python nous savions mieux comment nous organiser pour ne pas saturer la mémoire de nos ordinateurs. Nous avons principalement utilisé la bibliothèque Scikit Learn en supplément des bibliothèques python standard.[Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011]
Le code que nous avons écrit étant relativement important nous avons du l'organiser dans plusieurs fichiers et sous fichiers pour plus de clareté (tous sont disponibles). Les points d'entrées dans notre travail sont les deux notebook l'un portant sur l'aspect extraction des features et l'autre sur l'aspect prédiction de l'état de sommeil d'après ces features. Ces deux notebook utilisent des fonctions outils réparties dans les autres fichiers qu'il n'est pas forcément nécessaire pour une compréhension globale du travail.
Nous aurions pu essayer d'intégrer le rapport au notebook que nous utilisions lors de nos expérimentations en détaillant chaque étape. Cela ne nous semblait pas être le choix le plus judicieux (étant données les manipulations que nous avons du faire pour gérer la quantité de données) et nous avons préféré séparer le rapport du code utilisé pour en réduire la taille et nous permettre d'être un peu plus clair dans sa présentation. Des bouts de codes sont intégrés à ce rapport pour l'illustrer mais il ne s'agit pas de celui que avons utilisé lors du challenge. Nous pouvons en revanche que vous encourager à y jetter un rapide coup d'oeil, les notebook sont nous le pensons assez lisible, pour vous faire une idée ! Bonne lecture ! 

## Description du problème à traiter

### Problème biologique

Courte description du problème biologique

![](fig/eeg.png)

### Etat de l'art
Citer qques articles pour montrer qu'on en a lu et que le sujet a déjà été abordé par d'autres
### Données proposées 
Inclure un descriptif des données + peut être quelques visualisations obtenues (typiquement les diagrammes en boite qui nous ont aidées à choisir les features)

## Méthode
Pour définir notre méthode de travail nous avons décomposé le travail initialement en plusieurs étapes que nous allons détailler ici. Après avoir importé les données et les avoir formatées de manière à pouvoir les manipuler aisément dans notre environnement d'expérimentation nous avons dans un premier temps créer un pipeline d'extraction et de calcul des features sur les données brut et dans un deuxième temps un pipeline d'apprentissage à partir de ces features au préable sélectionnées et transformées et de prédiction à partir des données de validation. 

### Calculs des features
Comme décrit précédemment nous disposions de données brutes sous la formes de séries temporelles qui sont des données en très grande dimension. De plus la valeur d'un point d'une série temporelle pris isolément n'apporte pas d'information utile. Il suffirait par exemple de décaller un enregistrement de quelques dixièmes de seconde pour que la valeur de tous les points change radicalement sans pour autant que la série temporelle ne change de nature. C'est donc les dynamiques d'ensemble, les relations entre chacun de ces points qui nous intéressent. A partir de là et de la revue de la littérature explicité plus haut que nous avons faite, il nous semblait évident de commencer par calculer un certain nombre de features explicitant différents aspect de ces dynamiques d'ensemble (amplitude, variance, analyse de spectre, corrélation...). 
En pratique (voir le fichier feature_extraction_pipeline.ipynb) nous avons reproduit un certain nombre de features explicitées dans la méta analyse de S. Motamedi-Fakhr et al. Pour l'impémentation Python nous nous sommes appuyé sur les bibliothèques TsFresh proposant nombres des fonctions utiles pour l'analyse de séries temporelles [https://tsfresh.readthedocs.io/en/latest/text/introduction.html] ainsi que des bibliothèques plus classiques comme Scipy [https://www.scipy.org/] pour l'analyse fréquentielle par exemple.
Voici des extraits du code utilisé pour le calculs de features. Dans cet exemple, on explicite le calcul d'une feature, la corrélation de Benford, sur toutes les données EEG et de position (x, y, z).

In [ ]:
# Des étapes telles que l'importation des données ont été omises

from tsfresh.feature_extraction.feature_calculators import benford_correlation

# On définit d'abord une fonction permettant de calculer la feature sur une série temporelle quelconque

def benford_corr(x):
    return benford_correlation(x)

class Benford_corr:
    # Classe outil nécessaire au bon fonctionnement des autres fonctions
    f = benford_corr
    n_outs = 1
    f_names = ["Corrélation_de_Benford"]
    
from tools.processing import extract_all_features

# On créer une liste contenant la liste des features que l'on veut calculer
feat_class = [Benford_corr]

# On détaille la liste des séries temporelles sur lesquelles on veut calculer cette feature
columns = ["eeg_1","eeg_2","eeg_3","eeg_4","eeg_5","eeg_6","eeg_7", "x", "y", "z"]

# Cette fonction applique la feature à l'ensemble des données choisies et la sauvegarde dans un fichier
extract_all_features(feat_class, data=d, save="ben_correlation.csv", col=columns)

### Sélection et traitement des features
Une fois les features calculées et sauvegardées dans des fichiers il nous suffit de les charger dans notre pipeline de prediction (voir fichier prediction_pipeline.ipynb). Nous procédons tout d'abord à une étape de mise à l'échelle des données en retirant la moyenne de chaque feature et en divisant par l'écart type. Cette étape est nécessaire pour l'utilisation de certains modèles comme les SVM qui supposent des features de moyennes nulles et d'écarts types unitaires.

Rq: Nous avons pris soin de séparer notre jeu de données d'entrainement en deux avec des données d'entrainement et des données de test. La selection de features n'est opérée qu'en considérant les données d'entrainement. Nous auroins pu aller plus loin en utilisant un processus de cross validation global (sélection de features + sélection du modèle) mais cela n'aurait pas été possible matériellement. Nous en sommes resté à des processus des cross-validations à chaque sous étape ce qui était déjà assez demandant en ressources de calcul.

In [ ]:
from sklearn.preprocessing import StandardScaler

# On commence par diviser entre données d'entrainement et de test
# (f_train contient l'ensemble des features précédemment calculées)

X_train, X_test, y_train, y_test = train_test_split(f_train, y, test_size = 0.3)

# Normalisation des données 

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Le nombre de features restant important il nous semblait nécessaire de pouvoir procéder à une étape de sélection de features. Plusieurs méthodes s'offraient alors à nous. Nous avons choisi de commencer par supprimer les features pour lesquelles la variance était nulle. C'est à dire supprimer les features constantes n'apportant aucune information. Nous aurions pu également décider de ne pas inclure les features pour lesquelles la variance était trop faible. Nous en sommes restés à la variance nulle.

In [ ]:
# X_train est l'objet contenant les données chargées (toutes les features calculées pour toutes les séquences)
from sklearn.feature_selection import VarianceThreshold

v = 0

slct_low_variance = VarianceThreshold(threshold=v)
X_train = slct_low_variance.fit_transform(X_train)

Ensuite ayant remarqué que beaucoup de features étaient correlées nous avons mis en place une étape nous permettant d'effectuer une sélection multivariée. Nous avons procédé à une sélection par élimination de feature récursive via l'utilisation d'un SVM linéaire régularisé en l1. Ce mode de régularisation ayant tendance à donner un poid nul à certaines features, il nous suffit de récursivement les enlever, réentrainer sur les features restantes, jusqu'à obtenir le nombre de features désirées. Pour déterminer ce nombre nous procédons à une cross validation (5-fold) en essayant d'optimiser le f1-score (sur les prédictions du SVM linéaire).
Voilà un extrait du code utilisé à cette étape.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer

scorer_f1 = make_scorer(f1_score, average="weighted")

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False)

rec_slct = RFECV(lsvc, scoring = scorer_f1)

X_train = rec_slct.fit_transform(X_train, y_train)

Nous tenons à préciser ici que nous avons envisagé d'autres méthodes de sélections de features, notamment une sélection univariée s'appuyant sur des statistiques de student ou du $\chi_{2}$. Nous ne les présentons pas ici car, comme nous le précisons plus tard, l'étape de sélection de features ne nous a pas permis de d'améliorer nos résultats, nous ne nous y attardons donc pas.  

### Sélection du modèle
Nous avons mis en place une stratégie nous permettant de tester plusieurs modèles facilement. Nous verrons dans les résultats qu'un modèle s'est finalement avéré bien plus performant que tous les autres. Pour déterminer les hupyer-paramètres les plus efficaces nous effectué une recherche par grille avec une cross-validation (5-fold) pour atténuer la fluctuation statistique. Cela nous permet de tester toutes les combinaisons de paramètres et de déterminer la plus efficace. Cette étape est très couteuse en temps de calcul reste limitée dans le nombre de valeurs qui peuvent être testées. Voilà un exemple dans le cadre de l'optimisation d'un modèle SVM. On peut déterminer grâce à cela que la combinaison de paramètres la plus pertinente dans ce cas est un noyeau linéaire, $C=1$, et pas de rééquilibrage des données (piste envisagée étant donné que chaque classe n'est pas représentée de la même manière). 

In [ ]:
from sklearn.svm import SVC

clf = SVC()

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

scorer_f1 = make_scorer(f1_score, average="weighted")

param_grid = [
  {"C": [0.001, 0.01, 0.1], "kernel":["linear", "poly", "rbf", "sigmoid"], "class_weight":[None, "balanced"]}
 ]

tuner = GridSearchCV(clf, param_grid, scorer_f1,  verbose=2)

res = tuner.fit(X_train, y_train)

### Stratégie d'expérimentation dans le cadre d'un challenge
Etant donné que le travail se déroulait dans le contexte d'un challenge nous avons adapté notre méthode en conséquence. Nous avons voulu mettre en place un pipeline de données, suivant les étapes décrites précédemment, fonctionnel dès le début en commençant par une approche naïve de chaque étape avec pour objectif d'affiner chacune des étapes au fur et à mesure pour améliorer nos résultats. Cela nous permettait d'avoir des résultats rapidement et d'itérer fréquement pour améliorer nos performances et notre classement progressivement. Nous nous sommes vite aperçu que la quantité à traiter et nos moyens matériels limités ne nous permettraient pas de n'utiliser qu'un seul script que nous executerions dans son ensemble à chaque fois. Par exemple le calcule de quelques features simples sur tous les EEG des données d'entrainement et d'évaluation nécéssitait au minimum 2h30/3h de calculs sur nos ordinateurs, les étapes de crossvalidation nous prenait également au moins 3h de calculs. Nous avons donc ségmenté le travail en deux, un script dédié au calculs des features que nous enregistrons sur des fichiers en local de manière à ne pas avoir à calculer la même features deux fois. Et un script dédié au modèle prédictif qui charge les features précédemment calculées et implémente toutes les étapes jusqu'à la production d'un fichier réponse directement chargeable sur Kaggle. Nous avons pris le temps de développer des fonctions outils modulables nous permettant d'implémenter rapidement une nouvelle features, de la calculer sur les données que nous choisissions et de la charger facilement pour l'associer aux autres features déjà calculées. Nous le précisons car ce fut une part non négligeable de notre travail, qui pourrait dans un contexte de vie réelle être publié et réutilisé par d'autres équipes de recherches par exemple. Enfin nous avons mis en place un système nous permettant d'utiliser des capacités de calcul plus importantes que nos ordinateurs via le service Google Colab. Cela nous a permis de réduire d'environ 30% le temps de calculs des features.

## Résultats et discussion
Nous allons détailler ici quelques résultats obtenus. Dans un premier temps nous expliciterons les features que nous avons choisi de calculer. Nous détaillerons ensuite le modèle le plus performant et essairons d'analyser rapidement les résultats qu'il nous a permis d'obtenir en y associant quelques figures. Vous remarquerez que la partie sur la sélection de features est absente. Cela s'explique par le fait que notre le modèle le plus performant, la classification par Radom Forest, s'est montré la plus efficace sans cette étape de sélection de feature. Ce n'est pas une caractéristique intrinsèque de ce modèle, il n'y a pas de raison que la sélection de feature ne soit pas utile dans ce cas, mais aucune de nos tentatives ne s'est avérée pertinente. 

### Description des features utilisées
Petite listes des grandes catégories de features que nous avons calculées et pourquoi celles ci.
...

...

...

Pour nous aider à visulaliser la distribution de certaines features nous avons développé des fonction des visualisation nous permettant d'obtenir des figures de ce genre

![](fig/boxplot.png)

### Modèle choisi et performances 
Comme nous l'avons annoncé précédemment le modèle qui s'est avéré le plus efficace dans notre cas est une méthode ensembliste : la méthode Random Forest. Ce modèle consiste à entrainer plusieurs arbres de décisions que l'on pourrait juger plus faibles dont les prédictions aggrégées seronts finalement plus robustes qu'un arbre plus complexe qui aurait tendance par exemple à sur-entrainer. Ce modèle est le premier que nous avons envisagé mais nous avons eu du mal à dépasser ses performances autrement qu'en augmentant le nombre de features que nous lui fournissions en entrée. Beaucoup d'hyper paramètres peuvent être optimisés, si bien qu'il nous était impossible de faire une recherche exhaustive en utilisant la méthode précédemment présentée. Le principal que nous voulions traiter était d'empêcher un sur entrainement trop important. En effet il est très aisé d'atteindre un F1-score maximal sur les données d'entrainement mais ce même score sur les données de test plafonnait aux alentours de 0.80 (sur 30% des données) et encore moins sur les données d'évaluation (autour de 0.67). Avec cette idée en tête nous avons ciblé quelques paramètres à optimiser, notamment le nombre de sous-modèle et le nombre minimal d'échantillon par noeud (pour éviter de créer des noeuds spécifiques pour un seul échantillon) et une. En examinant la matrice de confusions (voir plus loin) nous avons remarqué que le fait que les classes ne soient pas équilibrées posait un problème. La classe la moins représenté est la classe 1, si bien que les performances sur cette classe sont bien moins hautes que pour les autres. Nous avons donc inclu dans les paramètres à tester une option permettant de rééquilibrer les différentes classes. La recherche par grille nous a permis de conclure qu'il ne fallait pas trop contraindre notre modèle, les meilleurs résultats ont été obtenus lorsque que nous laissions un nombre de sous-modèles assez important (100) pas de restriction dans le nombre d'échantillon par noeud et pas de rééquilibrage de classe. Finalement c'est comme si le sur entrainement n'est pas un problème si important et que c'est en sur entrainant complètement le modèle que l'on obtient les meilleurs résultats (notre objectif dans ce travail). Un petit commentaire ici. Etant donné que nos résultats sont encore inférieurs sur le set d'évaluation, le sur entrainement doit tout de même être un problème qu'il faudrait régler pour obtenir une meilleur généralisation sur ce set. Ce phénomène est à notre avis du au fait que les set d'entrainement et d'évaluation ne sont pas prélevés sur les mêmes individus, et nous l'aurions moins observé si toutes les séquences, de tous les individus, avaient été mélangées puis séparées aléatoirement en deux set. Un travail pourrait être fait pour homogénéiser ces deux sets, nous avons rapidement essayé mais devant l'absence de résultats prometteurs nous ne sommes pas allés plus loin. 
Voici un extrait du code que nous utilisons pour entrainer notre modèle.

In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100, 
                             criterion='gini', 
                             max_depth=None, 
                             min_samples_leaf=0.0001, 
                             class_weight="balanced",
                             bootstrap = False)

clf.fit(X_train, y_train)


predictions_train = clf.predict(X_train)
print(f"Training score - {f1_score(predictions_train, y_train, average='weighted')}")

# > 

predictions_test = clf.predict(X_test)
print(f"Testing score - {f1_score(predictions_test, y_test, average='weighted')}")

# > 

Voici également une matrice de confusion typique sur les données de test (X_test).

In [ ]:
from sklearn.metrics import confusion_matrix
from seaborn import heatmap

cm = confusion_matrix(y_test, predictions_test, normalize="true")
ax = heatmap(cm, annot=True)

![](fig/cf_matrix.png)

## Conclusion

Finalement comme nous travaillons dans le cadre d'un challenge avec un temps limité à dédier à ce projet (étant donné le travail que nous devons fournir pour d'autres matières et d'autres projets) nous avons tout de même adopté une démarche nous permettant d'avoir rapidement des résultats compétitifs. Or il se trouve que la technique de Random Forest nous a permis d'avoir des résultats convainquants et qui s'amélioraient au fur et à meusure que nous lui ajoutions de nouvelles features. Nous avons tout de même tenté d'autres approches et essayé d'affiner notre méthode en essayant d'autres modèles plus parlants, en selectionnant plus habilement les features utilisées. Force est de constater que cela n'améliorait pas les résultats obtenu par une démarche plus "brutale" et moins explicite : celle de donner un maximum de features à un modèle Random Forest. C'est un peu décevant, mais l'exercice nous contraignait un peu à ce niveau là. Ceci-dit, nous pensons qu'une démarche plus fine porterait très probablement ses fruits, nous péchons aussi par manque de temps (et de capacités matérielles, les calculs prenant beaucoup de temps sur des données de cette taille). 
A ceci s'ajoute que la méthode par Random Forest souffre d'un effet boîte noire qui rend les mécanismes sous jacents difficiles à analyser et donc à optimiser.
Nous aurions pu aller vers des techniques deep learning pour l'extraction de features: on ne l'a pas fait, cela nous semblait moins pertinent au regard du cours que nous avions suivi, bien qu'il y ai de bonnes chances que les techniques d'encodage par réseaux de neurones puissent donner des résultats intéressants.

In [ ]:
['eeg_0_mean' 'eeg_0_log_energy' 'eeg_0_kurtosis' 'eeg_1_log_ampl'
 'eeg_1_mean' 'eeg_1_log_energy' 'eeg_1_kurtosis' 'eeg_2_log_ampl'
 'eeg_2_mean' 'eeg_2_log_energy' 'eeg_2_kurtosis' 'eeg_3_log_ampl'
 'eeg_3_mean' 'eeg_3_log_energy' 'eeg_3_kurtosis' 'eeg_4_log_ampl'
 'eeg_4_mean' 'eeg_4_log_energy' 'eeg_4_kurtosis' 'eeg_5_log_ampl'
 'eeg_5_mean' 'eeg_5_log_energy' 'eeg_5_kurtosis' 'eeg_6_log_ampl'
 'eeg_6_mean' 'eeg_6_log_energy' 'eeg_6_kurtosis' 'eeg_0_skewness'
 'eeg_0_log_variance' 'eeg_0_zero_crossings' 'eeg_1_skewness'
 'eeg_1_log_variance' 'eeg_1_zero_crossings' 'eeg_2_skewness'
 'eeg_2_zero_crossings' 'eeg_3_skewness' 'eeg_3_log_variance'
 'eeg_3_zero_crossings' 'eeg_4_skewness' 'eeg_4_log_variance'
 'eeg_4_zero_crossings' 'eeg_5_skewness' 'eeg_5_log_variance'
 'eeg_5_zero_crossings' 'eeg_6_skewness' 'eeg_6_log_variance'
 'eeg_6_zero_crossings' 'pulse_log_ampl' 'pulse_log_variance' 'x_log_ampl'
 'x_log_variance' 'y_log_ampl' 'y_log_variance' 'z_log_ampl'
 'z_log_variance' 'eeg_1_band_theta' 'eeg_1_band_alpha' 'eeg_1_band_beta'
 'eeg_2_band_delta' 'eeg_2_band_theta' 'eeg_2_band_alpha'
 'eeg_2_band_beta' 'eeg_3_band_delta' 'eeg_3_band_theta' 'eeg_3_band_beta'
 'eeg_4_band_delta' 'eeg_4_band_theta' 'eeg_4_band_alpha'
 'eeg_5_band_theta' 'eeg_6_band_alpha' 'eeg_7_band_delta'
 'eeg_7_band_theta' 'eeg_7_band_alpha' 'eeg_7_band_beta'
 'eeg_1_fft_centroid' 'eeg_1_fft_variance' 'eeg_1_fft_skew'
 'eeg_1_fft_kurtosis' 'eeg_2_fft_centroid' 'eeg_2_fft_variance'
 'eeg_2_fft_skew' 'eeg_2_fft_kurtosis' 'eeg_3_fft_centroid'
 'eeg_3_fft_variance' 'eeg_3_fft_skew' 'eeg_3_fft_kurtosis'
 'eeg_4_fft_centroid' 'eeg_4_fft_variance' 'eeg_4_fft_skew'
 'eeg_4_fft_kurtosis' 'eeg_5_fft_centroid' 'eeg_5_fft_variance'
 'eeg_5_fft_skew' 'eeg_5_fft_kurtosis' 'eeg_6_fft_centroid'
 'eeg_6_fft_variance' 'eeg_6_fft_skew' 'eeg_6_fft_kurtosis'
 'eeg_7_fft_centroid' 'eeg_7_fft_variance' 'eeg_7_fft_skew'
 'eeg_7_fft_kurtosis' 'eeg_1_complexity' 'eeg_2_complexity'
 'eeg_3_complexity' 'eeg_4_complexity' 'eeg_5_complexity'
 'eeg_6_complexity' 'eeg_7_complexity' 'pulse_fft_centroid'
 'pulse_fft_variance' 'pulse_fft_skew' 'pulse_fft_kurtosis'
 'x_fft_centroid' 'x_fft_variance' 'x_fft_skew' 'x_fft_kurtosis'
 'y_fft_centroid' 'y_fft_variance' 'y_fft_skew' 'y_fft_kurtosis'
 'z_fft_centroid' 'z_fft_variance' 'z_fft_skew' 'z_fft_kurtosis']